This contains all the rough works as it is only for exploring the full data 

In [1]:
import geopandas as gpd
from get_metadata import get_data_from_date_range, get_data_by_country
from shapely import wkt
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import requests
import os
import rasterio
import numpy as np

#import the libraries

In [ ]:
gdf = gpd.read_file("WildflowerBlooms_AreaOfInterest.geojson")
BUCKET_NAME = 'radarsat-r1-l1-cog' #aws cli
s3client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

#Antelope valley from gdf just an example
antelope_valley_wkt = 'MULTIPOLYGON (((-118.50266751919736 34.69779051559962, -118.50230233686472 34.774843987788685, -118.50230233686472 34.774843987788685, -118.29013140159532 34.77593953478663, -118.28757512526677 34.620737043410564, -118.41831040035531 34.66748038198971, -118.50266751919736 34.69779051559962)))'

#sample data
sample = get_data_from_date_range( #march to may cuz spring
    start_year = 2010,
    start_month = 3,
    end_year = 2010,
    end_month = 5, 
    to_csv=True
)
# The get_metadata.py has a mistake as df.append() has been deprecated in the newer pandas so I edited that part

In [3]:
#read the data
df = pd.read_csv("sample.csv")
print(df.columns)

Index(['Unnamed: 0', 'product-type', 'start-date', 'product-format',
       'scene-centre', 'pixel-spacing', 'sensor', 'incidence-angle-high',
       'image-pixels', 'sensor-mode', 'orbit-direction', 'polarization',
       'corner-coordinates', 'processing-level', 'processor-version',
       'product-id', 'public-good', 'image-lines', 'end-date',
       'absolute-orbit', 'beam', 'delivery-date', 'processing-date', 'band',
       'spatial-resolution', 'order-key', 'position', 'sequence-id', 'title',
       'archive-visibility-start-date', 'incidence-angle-low',
       'geodetic-terrain-height', 'processor-name', 'look-orientation',
       'lut-applied', 'download_link'],
      dtype='object')


In [4]:
essential_columns = [
    'start-date', 'scene-centre', 'polarization', 'sensor-mode',
    'incidence-angle-low', 'incidence-angle-high','product-type'
]
filtered_df = df[essential_columns]
print(filtered_df.groupby(['polarization']).count())

              start-date  scene-centre  sensor-mode  incidence-angle-low  \
polarization                                                               
HH                   687          1182          687                  687   

              incidence-angle-high  product-type  
polarization                                      
HH                             687          1182  


Thats weird, it seems like radarsat1 only uses HH polarization, I will check the relationship between product-type and polarization

In [5]:
print(filtered_df.groupby(['product-type', 'polarization']).size())

product-type  polarization
SCN           HH              314
SCW           HH              648
SGF           HH              192
SGX           HH               28
dtype: int64


After doing a bit of research, We will be able to calculate the backscatter, for that we need to find the calibration constant K,                                                                                                                            I will use some tif files to clculate the calibration constant 

In [6]:
product_types = df['product-type'].unique()
print("Product types:", product_types) # will check the tif image from each productype and determine the K value

Product types: ['SCW' 'SCN' 'SGF' 'SGX']


In [8]:
def find_k_for_product_type(sample_urls):
    """Checks K constant for a few samples from each product type"""
    k_values = {}
    
    for product_type, url in sample_urls.items():
        try:
            with rasterio.open(url) as src:
                # Look for K in metadata
                tags = src.tags()
                band_tags = src.tags(1)
                
                print(f"\n=== {product_type} ===")
                print("Main tags:", tags)
                print("Band tags:", band_tags)
                
                # Try to find K - add common tag names here
                for tag_source in [tags, band_tags]:
                    for key, value in tag_source.items():
                        if any(kw in key.upper() for kw in ['CALIB', 'K', 'CONSTANT', 'FACTOR']):
                            print(f"FOUND POSSIBLE K: {key} = {value}")
                            k_values[product_type] = value
                            break
                
        except Exception as e:
            print(f"Error with {product_type}: {e}")
    
    return k_values

sample_urls = { # Downloaded in PC
    'SCN': 'https://s3-ca-central-1.amazonaws.com/radarsat-r1-l1-cog/2010/3/RS1_A0597112_SCNB_20100318_140941_HH_SCN01F.tif',
    'SCW': 'https://s3-ca-central-1.amazonaws.com/radarsat-r1-l1-cog/2010/3/RS1_A0596612_SCWB_20100301_220518_HH_SCW01F.tif',  # Replace with actual
    'SGF': 'https://s3-ca-central-1.amazonaws.com/radarsat-r1-l1-cog/2010/3/RS1_A0596882_S7_20100317_130044_HH_SGF01F.tif',
    'SGX': 'https://s3-ca-central-1.amazonaws.com/radarsat-r1-l1-cog/2010/3/RS1_P0597060_S2_20100316_213055_HH_SGX.tif'  
}

k_constants = find_k_for_product_type(sample_urls)
print("\nFINAL K CONSTANTS:", k_constants)


=== SCN ===
Main tags: {'CEOS_ACQUISITION_TIME': '20100318140938064               ', 'CEOS_ASC_DES': 'DESCENDING      ', 'CEOS_CALIBRATION_OFFSET': '0.0000000E+00', 'CEOS_ELLIPSOID': 'WGS84           ', 'CEOS_EPH_ORB_DATA_0': '7.1670660E+06', 'CEOS_EPH_ORB_DATA_1': '9.4314775E-05', 'CEOS_EPH_ORB_DATA_2': '-1.1509999E-03', 'CEOS_EPH_ORB_DATA_3': '1.1586051E+00', 'CEOS_EPH_ORB_DATA_4': '9.1803633E-02', 'CEOS_EPH_ORB_DATA_5': '1.4940349E+00', 'CEOS_EPH_ORB_DATA_6': '2.0000000E+00', 'CEOS_FACILITY': 'RSI             ', 'CEOS_GROUND_TO_SLANT_C0': '9.0650444E+05', 'CEOS_GROUND_TO_SLANT_C1': '5.0042295E-01', 'CEOS_GROUND_TO_SLANT_C2': '4.8255231E-07', 'CEOS_GROUND_TO_SLANT_C3': '-2.8114184E-13', 'CEOS_GROUND_TO_SLANT_C4': '7.9302200E-20', 'CEOS_GROUND_TO_SLANT_C5': '0.0000000E+00', 'CEOS_INC_ANGLE': '39.202', 'CEOS_LINE_SPACING_METERS': '25.0000000', 'CEOS_LOGICAL_VOLUME_ID': 'RSAT-1-SAR-SCN  ', 'CEOS_MISSION_ID': 'RSAT-1          ', 'CEOS_ORBIT_NUMBER': '75003', 'CEOS_PIXEL_SPACING_METERS':

That seems surprising, lets investigate more on why K is 0 even though it isnt supposed to be


In [12]:
from io import BytesIO

# --------- CONFIG: put your URL here ----------
url = "https://s3-ca-central-1.amazonaws.com/radarsat-r1-l1-cog/2010/5/RS1_G9599057_SCNA_20100517_113949_HH_SCN01F.tif"
# ----------------------------------------------

CALIB_KEYS = ['CALIB','CALIBRATION','CALFACTOR','CONVERSION','SIGMA','RCS',
              'RADIOMETRIC','ABSOLUTE','GAIN','SCALE','FACTOR','BETA','DB']
META_EXTS = ['.xml', '.ann', '.hdr', '.meta', '.txt', '.xml.gz']

print("Opening:", url, "\n")

# 1) Print embedded TIFF tags (filtered)
try:
    with rasterio.open(url) as src:
        tags = src.tags()
        band_tags = src.tags(1)
        print(">>> Embedded TIFF tags (matching calibration keywords or numeric hints):\n")
        for d in (tags, band_tags):
            for k, v in d.items():
                kup = k.upper()
                # Print tag if key looks calibration-y OR value looks numeric (not trivial 0/1)
                numeric_hit = False
                try:
                    f = float(v)
                    if f not in (0.0, 1.0) and abs(f) < 1e7:
                        numeric_hit = True
                except:
                    pass
                if any(kw in kup for kw in CALIB_KEYS) or numeric_hit:
                    print(f"  {k} = {v}")
except Exception as e:
    print("Error opening scene with rasterio:", e)

# 2) Try companion metadata files next to the COG URL
base = url.rsplit('.', 1)[0]
print("\nChecking companion metadata URLs near the COG (short snippets shown if found)...\n")
found_any = False

for ext in META_EXTS:
    meta_url = base + ext
    try:
        r = requests.get(meta_url, timeout=8)
    except Exception as e:
        print(f"  Could not fetch {meta_url}: {e}")
        continue

    if r.status_code == 200:
        # decode .gz if needed
        text = None
        if ext.endswith('.gz'):
            try:
                text = gzip.decompress(r.content).decode('utf-8', errors='ignore')
            except Exception:
                # fallback
                text = r.content.decode('utf-8', errors='ignore')
        else:
            text = r.text

        up = text.upper()
        hits = [kw for kw in CALIB_KEYS if kw in up]
        if hits:
            print(f"FOUND keywords {hits} in {meta_url}")
            snippet = text[:800].replace('\n', ' ')  # short single-line snippet
            print("  snippet:", snippet[:600], "...\n")
            found_any = True
        else:
            print(f"Checked {meta_url} — no obvious calibration keywords found (HTTP 200).")
    else:
        # optional: comment out to reduce noise
        print(f"Checked {meta_url} — HTTP {r.status_code}")

if not found_any:
    print("\nNo obvious calibration keywords found in companion files. That's OK — try the relative / z-score method or search full product folder on disk for CEOS/annotation files.")
else:
    print("If calibration lines were found above, inspect the snippet for numeric constants (CF/K).")

Opening: https://s3-ca-central-1.amazonaws.com/radarsat-r1-l1-cog/2010/5/RS1_G9599057_SCNA_20100517_113949_HH_SCN01F.tif 

>>> Embedded TIFF tags (matching calibration keywords or numeric hints):

  CEOS_CALIBRATION_OFFSET = 0.0000000E+00
  CEOS_EPH_ORB_DATA_0 = 7.1670715E+06
  CEOS_EPH_ORB_DATA_1 = -9.4544934E-04
  CEOS_EPH_ORB_DATA_2 = -6.7466480E-04
  CEOS_EPH_ORB_DATA_3 = 6.7447561E-01
  CEOS_EPH_ORB_DATA_4 = -9.4644409E-01
  CEOS_EPH_ORB_DATA_5 = 2.5247555E+00
  CEOS_EPH_ORB_DATA_6 = 2.0000000E+00
  CEOS_GROUND_TO_SLANT_C0 = 8.3512556E+05
  CEOS_GROUND_TO_SLANT_C1 = 3.1166485E-01
  CEOS_GROUND_TO_SLANT_C2 = 6.2132381E-07
  CEOS_GROUND_TO_SLANT_C3 = -2.8043751E-13
  CEOS_GROUND_TO_SLANT_C4 = 2.4311791E-20
  CEOS_INC_ANGLE = 30.172
  CEOS_LINE_SPACING_METERS = 25.0000000
  CEOS_ORBIT_NUMBER = 75859
  CEOS_PIXEL_SPACING_METERS = 25.0000000
  CEOS_PLATFORM_HEADING = -0.058
  CEOS_PLATFORM_LATITUDE = 45.896
  CEOS_PLATFORM_LONGITUDE = -76.558
  CEOS_SCENE_LENGTH_KM = 353.4249878
  CEOS

I got an undestanding of the data but I give up and will explore EODMS

Seems like I have been wasting my time, there is an API for this whole thing